# 協調フィルタリングを用いた商品リコメンド 

協調フィルタリングというアルゴリズムを使って、ユーザーにおすすめの商品を紹介する機能を作ります。

教材は[/Advance/algorithm/recommend](https://techpreneur-dojo2021.growi.cloud/Advance/algorithm/recommend) にあります。協調フィルタリングの説明などはこちらをご覧ください。


In [ ]:
# 必要なモジュールをインポート
!pip install numpy
!pip install pandas
!pip install scikit-learn


^C


You should consider upgrading via the 'c:\users\owner\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\owner\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\owner\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances

# データをダウンロード
[Google Drive](https://drive.google.com/drive/folders/1zZrvcj2SSy2y6e7Ii4Z7i2T4_qcIR2gj)から学習用データをダウンロードします。
この方法ではなくても良いですが、このipynbファイル直下にdataフォルダを作り、
その中にCSVファイルを入れてください。
[https://gitlab.com/techpreneur/dojo-summer-2021/advance/algorithm/](algorithmレポジトリ)をクローンして、フォルダ内のbatファイルまたはshファイルを実行することでも同様のことができます。


In [2]:
# 必要なデータをダウンロード

!pip install wget
import wget
wget.download("https://docs.google.com/uc?export=download&id=18_r7IRfnN-lnlokbQZ6A4YCSBnEVOj3d", out = "./data/profile.csv")
wget.download("https://docs.google.com/uc?export=download&id=1SlSfD8GTzapn3wjgdL4BNAerYO8GeW0S", out = "./data/RentRoom.csv")
wget.download("https://docs.google.com/uc?export=download&id=1S_1IDepWMBPUaXiXezADwVnsUasHPZ9i", out = "./data/cross.csv")
wget.download("https://docs.google.com/uc?export=download&id=1SkbuZObukLsF2nWxlAv3Xa5ueY3zrXKj", out = "./data/scoreLog.csv")
wget.download("https://docs.google.com/uc?export=download&id=1Ub0MH-FCzRHzMATprw7KojReVUU2q0DS", out = "./data/station.csv")

FileNotFoundError: [Errno 2] No such file or directory: '.\\./data/profile.csvgexcdfk0.tmp'

In [ ]:
#  "profile.csv", "RentRoom.csv", "cross.csv", "scoreLog.csv", "station.csv"の
# 5つのファイルがdataフォルダの中にあることを確認します。

# Mac, Linuxユーザー用？(未確認)
!ls ./data

# Windowsユーザー用
%cd ./data
%ls
%cd ..

C:\Users\Owner\Desktop\temp2\algorithm\data


'ls' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


 ドライブ C のボリューム ラベルは Windows です
 ボリューム シリアル番号は 408B-8E11 です

 C:\Users\Owner\Desktop\temp2\algorithm\data のディレクトリ

2021/08/26  10:51    <DIR>          .
2021/08/26  10:51    <DIR>          ..
2021/08/23  15:27           798,792 cross.csv
2021/08/23  14:42             1,000 data_downloader.sh
2021/08/26  10:51             1,428 profile (1).csv
2021/08/23  15:27             1,428 profile.csv
2021/08/26  10:51           445,207 RentRoom (1).csv
2021/08/23  15:27           445,207 RentRoom.csv
2021/08/23  15:27         1,800,040 scoreLog.csv
2021/08/23  15:27             1,606 station.csv
               8 個のファイル           3,494,708 バイト
               2 個のディレクトリ  59,199,516,672 バイトの空き領域
C:\Users\Owner\Desktop\temp2\algorithm


## データの読み込み

先ほどダウンロードしたクロス集計表データcross.csvをダウンロードします。

cross.csvは以下の様なデータ形式をしており、全ユーザの全商品に対する5段階評価（1～5の整数で表す）を格納しています（未評価は0）

↓cross.csvの一部

|id,     |age, |gender, |household_num, |A00001, |A00002, |A00003|......|
|  --    | -   | -      |     -         | - | - | -  | -  |
|B00001, |62,  |1,      |1,             |0,      |0,       |3, |.......|
|B00002. |35,  |0,      |1,             |3,      |0,       |1,| ....... |


In [ ]:
df_temp = pd.read_csv("./data/cross.csv")
scores = np.array(df_temp.iloc[:,4:]) # 各アイテムの評価（5列目以降）のみをnumoy行列として保存
scores_T = scores.T
num_user = len(scores)
num_item = (scores.shape)[1]

In [ ]:
scores

array([[0, 0, 3, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
scores_T

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
print(num_user, num_item)

100 3852


## ユーザーベースのリコメンド

では、実際にリコメンド部分をプログラミングします。
ユーザーベースのリコメンドを一言で言うと、「***同じような商品評価傾向にあるユーザーが高く評価した商品をレコメンドすればよい***」

手順を簡単に説明すると、

1. ユーザー間の類似度を各ユーザーの商品評価（cross.csv）をもとに算出
    - 「調整コサイン類似度」を使用
1. リコメンド対象ユーザーと10個以上同じ商品を評価しているユーザーに対してのみ抜き出し、1.で計算したリコメンド対象ユーザーとの類似度が大きい順に並べる
1. 類似度順ユーザーリストを基に、リコメンド対象ユーザー が商品に対して付けうる評価　
を予測し、それをすべての商品に対して行う。 
1. 3.で計算した商品予測評価値が大きい順に並べたものがリコメンド

### 1.＆2. ユーザー間の類似度を算出
- コサイン類似度とは、2つのベクトルのなす角のコサインを表す
- $cos(\overrightarrow{a},\overrightarrow{b}) = \dfrac{\overrightarrow{a}\cdot\overrightarrow{b}}{|\overrightarrow{a}||\overrightarrow{b}|}$、$-1 \leqq cos(\overrightarrow{a},\overrightarrow{b}) \leqq 1$
- つまり、二つのベクトルの方向が似ているほどコサイン類似度の値は大きくなり1に近づく
- ユーザー $i$ とユーザー $j$ の類似度は、そのユーザの商品評価をベクトルとしてあらわしたもの（$(r_{(u_1, i)}, r_{(u_2, i)}, ...., r_{(u_n, i)})$ と、$(r_{(u_1, j)}, r_{(u_2, j)}, ...., r_{(u_n, j)})$）のコサイン類似度で表すことができます。
    - →コサイン類似度が高い（1に近い）＝ユーザー$i$とユーザー$j$は類似度が高い！
- そして、これを改良したものが「調整」コサイン類似度で、以下の式で表される。
- $sim(i,j) = \dfrac{\sum_{u \in U_i \cap U_j} (r_{ui}-\bar{r_u})\cdot(r_{uj}-\bar{r_u})}{\sqrt{\sum_{u \in U_i \cap U_j}(r_{ui} - \bar{r_u})^2} \cdot \sqrt{\sum_{u \in U_i \cap U_j}(r_{uj} - \bar{r_u})^2}}$
    - $r_{(u,i)}$ はユーザ $u$ のアイテム $i$ に対する評価点、 
    - $\bar{r_u}$ はユーザ $u$ の全アイテムの評価点の平均とします。
- Pythonでは`sklearn.metrics.pairwise`のなかの`pairwise_distances`関数で調整コサイン類似度を計算できる

In [ ]:
# ユーザー間の類似度を算出

def get_sim_user(target_userid):
    global df_temp, scores, scores_T, num_user, num_item

    #コサイン類似度の計算
    sim_mt = 1 - pairwise_distances(scores, metric='cosine')
    
    print("#######################  sim_mt : #######################")
    print(sim_mt) # sim_mt[i][j]はユーザーiとユーザーjのコサイン類似度を表す
    print(np.shape(sim_mt)) # sim_mtの行列のサイズを表示 -> 100x100
    print("#########################################################")

    #評価マトリクスと類似度マトリクスから、指定ユーザーの記録を抽出
    score_target = scores[target_userid] # [ r_0, r_1, ..., r_n ] target_useridの全商品の評価
    sim_target = sim_mt[target_userid] # 

    #評価物件の共通件数を計算し、10件以上ある場合に参照する
    def check_valid_user(i):
        indices = np.where(((score_target) * (scores[i])) != 0)[0]
        return len(indices) > 10 and i != target_userid

    similarities = [] # (ユーザーID, 類似度)のリスト
    for i in range(num_user):
        if check_valid_user(i): # 10件以上共通して評価している商品があるか？
            similarities.append((i, sim_target[i]))
        else:
            similarities.append((i, 0)) # 使えないデータとして使用しない（類似度0とする）
    
    # ↑のsimilaritiesの6行は、以下の様にシンプルに書くこともできます。
    similarities = [ 
      ((i, sim_target[i]) if check_valid_user(i) else (i, 0)) 
      for i in range(num_user)
    ]
            
    #降順に並び替えて返す
    return sorted(similarities, key=lambda s: s[1], reverse=True)

sim = get_sim_user(10) # ユーザーID10との類似度の高いユーザーを順番に出力
print("上位五名の類似ユーザーと類似度＝", sim[0:5])


#######################  sim_mt : #######################
[[1.         0.08990943 0.1113774  ... 0.1053935  0.12061624 0.10482957]
 [0.08990943 1.         0.11408457 ... 0.09812849 0.10144441 0.13163438]
 [0.1113774  0.11408457 1.         ... 0.08843851 0.0914686  0.08637074]
 ...
 [0.1053935  0.09812849 0.08843851 ... 1.         0.09673566 0.09983056]
 [0.12061624 0.10144441 0.0914686  ... 0.09673566 1.         0.09543552]
 [0.10482957 0.13163438 0.08637074 ... 0.09983056 0.09543552 1.        ]]
(100, 100)
#########################################################
上位五名の類似ユーザーと類似度＝ [(91, 0.12767238965361927), (55, 0.12354445837537187), (37, 0.12243713683101065), (4, 0.12101778409367125), (5, 0.119278599250077)]


### 3.一つの商品に対するレコメンド


作成した類似度順ユーザーリストを基に、*リコメンド対象ユーザー* が商品$i$に対して付けうる評価　$r'_{u, i} $ を予測し、それをすべての商品に対して行う。
- 評価予測は以下の式で行う：
- $r'_{u, i} = \dfrac{\sum_{j \in Q_t(u)} sim(u, j) * r_{j, i}}{\sum_{j \in Q_t(u)} sim(u, j)}$
  - ユーザ$u$の未評価アイテム$i$に対する予測評価点$r'_{(u, i)}$を求める
  - $r_{u, i} = \sum_{j \in Q_t(u)} sim(i, j) * r_{u, j}$とする
  - $Q_{t(u)}$は、ユーザー$u$と一定以上類似度が高いユーザーの集団（今回は上位5名）
- ユーザー$j$の商品$i$に対する評価$r_{j, i}$を、類似度の高いユーザーそれぞれで合計しているが、そのときにユーザー$j$とリコメンド対象ユーザー* $u$ の類似度を重みとして掛け算した和
    - 類似度の高いユーザほど、その人の評価値をリコメンドに大きく反映させる

In [ ]:
# 次に、上で定義した類似度データを用いて、
# ユーザーtarget_useridの商品target_item_idxに対する
# おすすめ度を推測する関数を定義します。
# 引数kは類似度の高いユーザー評価を何人分利用するかを設定します。

def predict_user(similarities, target_userid, target_item_idx, k=8):
    global df_temp, scores, scores_T, num_user, num_item
    
    # リコメンド対象ユーザの今までの評価の平均(avg_target)を、最終的な予測評価値にプラスする用
    target_score = scores[target_userid]
    avg_target = np.mean(target_score[np.where(target_score >= 1)]) 

    numerator = 0.0
    denominator = 0.0
    k_tmp = 0

    for similarity in similarities: # 全てのユーザーに対して, 類似度が高い順にみていく
        s_usr   = similarity[0] # 類似しているユーザーID
        s_sim = similarity[1] # ↑s_usr変数と、リコメンド対象ユーザの類似度

        if k_tmp > k or s_sim <= 0.0: break # 類似度の高いユーザk件のみを評価する
        score = scores[s_usr]
        if score[target_item_idx] >= 1: # そのユーザーがtarget_item_idxの商品を評価している時
            denominator += s_sim
            numerator += s_sim * (score[target_item_idx] - np.mean(score[np.where(score >= 1)]))
            k_tmp += 1
    return avg_target + (numerator / denominator) if denominator > 0 else 0

print("ユーザーID10の、商品ID30に対する予測評価は",predict_user(sim, 10, 30), "です")


ユーザーID10の、商品ID30に対する予測評価は 1.874346510225877 です


### 4. ↑の手順3.を全商品に対して行いリコメンド商品を決定

In [ ]:
# ↑のpredict_user関数を全ての商品に対してfor文で回せばリコメンドの完成です。

def rank_items_user( similarities, target_userid):
    global df_temp, scores, scores_T, num_user, num_item
    rankings = []
    target_score = scores[target_userid]
    # 物件全てで評価値を予測
    for i in range(num_item):
        # 既に評価済みの場合はスキップ
        if target_score[i] >= 1:
            continue
        #関数参照のためself挿入
        rankings.append((i, predict_user(similarities, target_userid, i)))
    return sorted(rankings, key=lambda r: r[1], reverse=True)

ranking = rank_items_user(sim, 10)
ranking[0:10] # 上位10個のおすすめ商品を表示（商品ID, 予測評価）の順）

[(1145, 4.784249184013424),
 (3221, 4.610787130186826),
 (1196, 4.500085374870782),
 (2306, 4.4563769171531895),
 (1491, 4.429958411632503),
 (2744, 4.41531372827862),
 (2711, 4.35645259353777),
 (222, 4.352502018885943),
 (1542, 4.309079731257668),
 (3685, 4.306113636613402)]

# アイテムベースのリコメンド

今度はユーザベースのリコメンドとは逆のことを行います。

***全ユーザの評価を基に、商品ごとの類似度を計算し、同じような類似度だったら評価値も同じような値になるだろう***という考え方

手順は

1. 調整コサイン類似度を用いて、各ユーザーの商品評価の値をもとに、各商品どうしの類似度を計算します。
2. 以下の式で商品$i$のユーザー$u$の評価予測値を算出する。
	- $r'_(i, u) = \dfrac{\sum_{j \in Y(u)} sim(i, j) \cdot score(u, j)}{\sum_{j \in Y(u)} sim(i, j)}$
	- $Y(u)$はユーザー$u$が評価している商品のリスト
	- $score(u, j)$は、ユーザー$u$の商品$j$に対する評価点
	- 類似度が高い商品の場合、評価点数も同じになるだろうという考えで作成されたもの
	- ユーザー$u$の評価点数に商品ごとの類似度という重みを付けた和
3. すべての商品について2.を繰り返し行い、2.の値が高いものから順にレコメンドする。




In [ ]:
# 調整コサイン類似度を用いて、商品同士の類似度算出
# get_sim_item関数の返り値は  商品数 x 商品数の行列で、sim[i, j]は商品iとjの類似度を表す
# つまり、返り値は対称行列(sim[i, j] = sim[j, i])になっている

def get_sim_item():
    global df_temp, scores, scores_T, num_user, num_item
    similarities = 1 - pairwise_distances(scores_T, metric='cosine')
    # np.fill_diagonal(similarities, 0) # 体格成分を0にする
    return similarities
sim = get_sim_item()
print(sim)
sim.shape

[[1.         0.         0.14440527 ... 0.21798446 0.         0.0732645 ]
 [0.         1.         0.         ... 0.08097763 0.         0.        ]
 [0.14440527 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.21798446 0.08097763 0.         ... 1.         0.09338049 0.        ]
 [0.         0.         0.         ... 0.09338049 1.         0.15064872]
 [0.0732645  0.         0.         ... 0.         0.15064872 1.        ]]


(3852, 3852)

In [ ]:
def rank_items_item(similarities, target_userid):
    global df_temp, scores, scores_T, num_user, num_item
    u = scores[target_userid] # target_useridの全商品に対する評価値リスト

    # xは商品数 x 商品数の配列で、
    # リコメンド対象ユーザが評価済みの商品jの部分(x[j])だけ値を1にする
    x = np.zeros(num_item)
    x[u > 0] = 1 

    nominater = similarities.dot(u)
    denominater = similarities.dot(x)

    rankings = [(i, 0) for i in range(num_item)]

    for i in range(num_item):
        if denominater[i] != 0 and u[i] > 0:
            rankings[i] = (i, nominater[i] / denominater[i])
    return sorted(rankings, key=lambda r: r[1], reverse=True)

rankings = rank_items_item(sim, 10) # ユーザー10に対するリコメンド
print("おすすめ商品上位5個は", rankings[0:5], "です")

おすすめ商品上位5個は [(1462, 3.4989695417592994), (896, 3.445390426682878), (2341, 3.431449194547922), (2224, 3.4253902133319536), (1821, 3.424193347885699)] です


# 新規ユーザのリコメンド
新規ユーザの場合は、過去の商品評価データがないため、ユーザや商品の類似度を算出して,,,ということができません。
「最寄り駅」と「年齢」の情報だけからリコメンデーションをする必要があります。

ということで、以下の様なアプローチを行います。

- 最寄り駅が一致している物件に絞り込む
- 同じような年齢層の既存ユーザーの評価を商品ごとに合計し、合計が高いものからリコメンデーションする
   - 今回は、「同じような年齢層」は「年齢の10の位が同じ人」として実装する


全体プログラム：https://gitlab.com/techpreneur/dojo-summer-2021/advance/algorithm/-/blob/main/6_recommend_new_user.py

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
# データを読みこみます。
df_estate = pd.read_csv("./data/RentRoom.csv")

# RentRoom.csvの中身
# id,     pref_name, city_name,  district_name, built_year, structure,  top_floor_num, room_type, \
# A00001, 東京都,     世田谷区,    喜多見７丁目,     1988,      ＲＣ,        2,              1R,  \
# price,   area,  latitude,    longitude,    nearest_station_id,  dist_to_nearest_station
# 2812.27,  23,   35.63129425, 139.5940049,  S00001,              1096.93

df_station = pd.read_csv("./data/station.csv")
# station.csvの中身
# id    name   latitude   longitude
# 0   S00001     喜多見  35.636745  139.587513
# 1   S00002    駒沢大学  35.633489  139.661627
# 2   S00003     上野毛  35.612023  139.638845
# 3   S00004   二子玉川園  35.611512  139.626545


df_user = pd.read_csv("./data/cross.csv")
# cross.csvの中身：
# id,     age, gender, household_num, A00001, A00002, A00003......
# B00001, 62,  1,      1,             0,      0,       3, .......
# B00002. 35,  0,      1,             3,      0,       1, ....... 

# ユーザデータに新しく`age_10`というカラムを作り、年齢（`age`カラムの値）の10の位を代入します
df_user["age_10"] = df_user["age"].map(lambda x: str(x)[:1]) 

In [ ]:
# 最寄り駅が同じ物件のみ絞り込みます。
# rentRoom.csvに入っている駅データはIDとして格納されています。
# しかし、入力フォームはIDではなく駅名を入力したいので、
# 絞り込むには駅名→IDに変換するget_station_id関数が必要です

def get_station_id(station_str):
    global df_estate, df_user, df_station
    df_station_extracted = df_station[df_station["name"] == station_str]
    if len(df_station_extracted) == 0:
        print("該当する駅が存在しませせん")
        print("駅リスト : ", df_station["name"])
        return ""
    station_id = str(df_station_extracted['id'].values[0])
    return station_id 

get_station_id("下北沢")

'S00038'

In [ ]:
# 年齢の差が10歳未満の人のみ絞り込む
age = 30 # ターゲットの年齢は30歳とする
df_user_extracted = df_user[abs(df_user["age"] - age) <= 10] 

# 下北沢駅のIDを取得
station_id = get_station_id("下北沢")

# df_extractedに最寄り駅が下北沢の物件のみ抽出
df_extracted = df_estate[df_estate["nearest_station_id"] == station_id]

# df_extractedのIDだけを取り出してリストとして取り出す
valid_estate_ids = df_extracted["id"].tolist()

# 年齢層が同じユーザの評価のみを取り出す
df_extracted = df_user_extracted[valid_estate_ids]

# 各物件に対して、同じ年齢層のユーザー評価の平均を取り、その値でソートしてリコメンド！
mean_scores = df_extracted.mean().sort_values(ascending=False).reset_index()
rankings = [list(mean_scores.loc[i]) for i in range(len(mean_scores))]

rankings[0:5]


[['A03809', 0.7575757575757576],
 ['A03834', 0.6666666666666666],
 ['A03830', 0.6060606060606061],
 ['A03817', 0.6060606060606061],
 ['A03821', 0.6060606060606061]]